In [1]:
!pip install transformers
!pip install sentencepiece
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!pip install simpletransformers

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 57.9 MB/s 
     |████████████████████████████████| 67 kB 7.4 MB/s 
     |████████████████████████████████| 596 kB 52.4 MB/s 
     |████████████████████████████████| 6.5 MB 58.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
     |████████████████████████████████| 248 kB 4.1 MB/s 
     |████████████████████████████████| 1.7 MB 67.0 MB/s 
     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 9.9 MB 77.1 MB/s 
     |████████████████████████████████| 311 kB 73.0 MB/s 
     |████████████████████████████████| 181 kB 78.6 MB/s 
     |████████████████████████████████| 144 kB 78.3 MB/s 
     |███████████████

In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os

In [3]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()
print('Cuda available? ',cuda_available)

# Get the GPU device name.
device_name = tf.test.gpu_device_name()
# The device name should look like the following:
if device_name == '/device:GPU:0':
  print('Found GPU at: {}'.format(device_name))
else:
  raise SystemError('GPU device not found')

Cuda available?  True
Found GPU at: /device:GPU:0


In [4]:
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [45]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

from dont_patronize_me import DontPatronizeMe

dpm = DontPatronizeMe('.', '.')
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label,
      'keyword':keyword
  })
trdf2 = pd.DataFrame(rows)

rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  orig_label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].orig_label.values[0]
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label,
      'orig_label': orig_label
  })
tedf2 = pd.DataFrame(rows)

# downsample negative instances
pcldf = pd.DataFrame(trdf2[trdf2.label==1])

print(len(pcldf))


Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py
Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}
794


In [46]:
#Data augmantation
replicated_pcls = pd.concat([pcldf] * 8, ignore_index=True)
print(len(replicated_pcls))

6352


In [47]:
npos = len(replicated_pcls)
#downsampling negative instances to 1:1
training_set2 = pd.concat([replicated_pcls,trdf2[trdf2.label==0][:int(npos)]])
#Data Sampling
training_set2 = training_set2.sample(frac = 1, random_state = 42)
#Size of final training dataset
print(len(training_set2))

12704


In [8]:
#This is for roberta-base
task1_model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = task1_model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(training_set2[['text', 'label']])
# run predictions
preds_task1, _ = task1_model.predict(tedf2.text.tolist())
#task1_model.train_model(training_set2[['text', 'label']], eval_df=tedf2[['text', 'label']])

#preds_task1,raw = task1_model.predict(tedf2.text.tolist())

preds_task1_bert = [[k] for k in preds_task1]
labels2file(preds_task1_bert, 'task1_res.txt')
labels2file([[k] for k in tedf2.label.values],'task1_ref.txt')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1588 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

In [9]:
#Evaluation on official dev dataset for roberta


from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
print('precision: ' + str(precision_score(tedf2.label.values, preds_task1)))
print( 'recall： ' + str(recall_score(tedf2.label.values, preds_task1)))
print('f1_score: ' + str(f1_score(tedf2.label.values, preds_task1)))


precision: 0.5095785440613027
recall： 0.6683417085427136
f1_score: 0.5782608695652174


In [10]:
# Analysis 3.1
res = []

level0df = pd.DataFrame(tedf2[tedf2.orig_label=='0'])
preds_task1_l0, _ = task1_model.predict(level0df.text.tolist())

f1_l0 = accuracy_score(level0df.label.values, preds_task1_l0)
print(f1_l0)
print(len(level0df))
res.append(f1_l0)


leveldf = pd.DataFrame(tedf2[tedf2.orig_label=='1'])
preds_task1_l, _ = task1_model.predict(leveldf.text.tolist())

acc_l = accuracy_score(leveldf.label.values, preds_task1_l)

print(acc_l)
print(len(leveldf))
res.append(acc_l)

leveldf = pd.DataFrame(tedf2[tedf2.orig_label=='2'])
preds_task1_l, _ = task1_model.predict(leveldf.text.tolist())

acc_l = accuracy_score(leveldf.label.values, preds_task1_l)

print(acc_l)
print(len(leveldf))
res.append(acc_l)

leveldf = pd.DataFrame(tedf2[tedf2.orig_label=='3'])
preds_task1_l, _ = task1_model.predict(leveldf.text.tolist())

acc_l = accuracy_score(leveldf.label.values, preds_task1_l)

print(acc_l)
print(len(leveldf))
res.append(acc_l)

leveldf = pd.DataFrame(tedf2[tedf2.orig_label=='4'])
preds_task1_l, _ = task1_model.predict(leveldf.text.tolist())

acc_l = accuracy_score(leveldf.label.values, preds_task1_l)

print(acc_l)
print(len(leveldf))
res.append(acc_l)



INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1704 [00:00<?, ?it/s]

  0%|          | 0/213 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


0.9507042253521126
1704


...

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


0.5955056179775281
89


  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

0.8043478260869565
92


In [11]:
# Analysis 3.2
import numpy as np


#print(training_set2.text.tolist)
avg_length = np.median([len(x) for x in training_set2.text.tolist()])
print('median length of training test: ' + str(avg_length))

#long_training_set = pd.DataFrame([])
long_training_set = []
short_training_set = []

for item in trdf2.values:
  if len(item[1]) > avg_length:
    
    long_training_set.append([item[1], item[2]])
  else:
    short_training_set.append([item[1], item[2]])
print(len(long_training_set))
print(len(short_training_set))

median length of training test: 243.0
3909
4466


In [42]:
#Training data with longer text
task1_model.train_model(pd.DataFrame(short_training_set).rename(columns={0: "text", 1: "label"}))
preds_task1, _ = task1_model.predict(tedf2.text.tolist())
print('accuracy: ' + str(accuracy_score(tedf2.label.values, preds_task1)))


/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/4466 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/559 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

accuracy: 0.9016236867239733


In [44]:
task1_model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = task1_model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
task1_model.train_model(pd.DataFrame(long_training_set).rename(columns={0: "text", 1: "label"}))
preds_task1, _ = task1_model.predict(tedf2.text.tolist())
print('accuracy: ' + str(accuracy_score(tedf2.label.values, preds_task1)))


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/3909 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/489 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

accuracy: 0.9164278892072588


In [55]:
#Analysis 3.3
category_list = set(training_set2.keyword.tolist())
print(category_list)

accl = {}
f1l = {}
for item in category_list:
  catdf = pd.DataFrame(training_set2[training_set2.keyword==item])

  print(len(catdf))

  task1_model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
  task1_model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = task1_model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
  # train model
  task1_model.train_model(catdf[['text', 'label']])
  # run predictions
  preds_task1, _ = task1_model.predict(tedf2.text.tolist())

  acc = accuracy_score(tedf2.label.values, preds_task1)
  f1 = f1_score(tedf2.label.values, preds_task1)
  print(item)
  print(acc)
  print(f1)

  accl[item] = acc
  f1l[item] = f1


{'refugee', 'hopeless', 'vulnerable', 'disabled', 'migrant', 'women', 'poor-families', 'in-need', 'immigrant', 'homeless'}
1254


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/1254 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

homeless
0.7550143266475645
0.37968561064087064


In [57]:
for item in category_list:
  print(item)
  print(f1l[item])

refugee
0.4652777777777778
hopeless
0.31844215349369986
vulnerable
0.4405940594059406
disabled
0.38549618320610685
migrant
0.4155844155844156
women
0.2721518987341772
poor-families
0.4052980132450331
in-need
0.3799019607843137
immigrant
0.37426900584795325
homeless
0.37968561064087064


In [ ]:
#We have also tried other model and techniques, but they don't improve the performance
#we have list their final parameter and evaluation result
#on official dev set down below

In [26]:
#This is for distilbert
task1_model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("distilbert", 
                                  'distilbert-base-uncased', 
                                  args = task1_model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(training_set2[['text', 'label']])
# run predictions
preds_task1, _ = task1_model.predict(tedf2.text.tolist())
#task1_model.train_model(training_set2[['text', 'label']], eval_df=tedf2[['text', 'label']])

#preds_task1,raw = task1_model.predict(tedf2.text.tolist())

preds_task1_bert = [[k] for k in preds_task1]
labels2file(preds_task1_bert, 'task1_res.txt')
labels2file([[k] for k in tedf2.label.values],'task1_ref.txt')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

  0%|          | 0/12704 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1588 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

In [27]:
#Evaluation on official dev dataset for distilbert


from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
print('precision: ' + str(precision_score(tedf2.label.values, preds_task1)))
print( 'recall： ' + str(recall_score(tedf2.label.values, preds_task1)))
print('f1_score: ' + str(f1_score(tedf2.label.values, preds_task1)))


precision: 0.515
recall： 0.5175879396984925
f1_score: 0.5162907268170427


In [21]:
#This is for xlnet
task1_model_args = ClassificationArgs(num_train_epochs=5, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True,
                                     train_batch_size=32,
                                     #evaluate_during_training = True,
                                     #evaluate_during_training_steps = 86,
                                     #evaluate_during_training_verbose = True,
                                     #use_cached_eval_features = True,
                                     #use_early_stopping = True,
                                     early_stopping_delta = 0.01,
                                     early_stopping_metric = "mcc",
                                     early_stopping_metric_minimize = True,
                                     early_stopping_patience = 2)

task1_model = ClassificationModel("xlnet", 
                                  'xlnet-base-cased', 
                                  args = task1_model_args, 
                                  num_labels=2,
                                  weight=[0.25,0.70],
                                  use_cuda=cuda_available)

task1_model.train_model(training_set2[['text', 'label']], eval_df=tedf2[['text', 'label']])

preds_task1,raw = task1_model.predict(tedf2.text.tolist())

preds_task1_bert = [[k] for k in preds_task1]
labels2file(preds_task1_bert, 'task1_res.txt')
labels2file([[k] for k in tedf2.label.values],'task1_ref.txt')

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

  0%|          | 0/12704 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/397 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/397 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/397 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of xlnet model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

In [23]:
#Evaluation on official dev dataset for xlnet


from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
print('precision: ' + str(precision_score(tedf2.label.values, preds_task1)))
print( 'recall： ' + str(recall_score(tedf2.label.values, preds_task1)))
print('f1_score: ' + str(f1_score(tedf2.label.values, preds_task1)))


precision: 0.6357142857142857
recall： 0.4472361809045226
f1_score: 0.5250737463126843


In [28]:
#Output testing result
df_test = pd.read_csv('task4_test.tsv', sep='\t', header=None, engine = 'python')
print(len(df_test))
df_test = df_test.drop(columns = [0])
df_test.columns = ['ID', 'keyword', 'country_code', 'paragraph']

3832


In [29]:
preds_task4, _ = task1_model.predict(df_test.paragraph.tolist())
labels2file([[k] for k in preds_task4], 'task1.txt')

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3832 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

In [ ]:
#Back Translation
for i in range(len(pcldf)):
  print(i)
  print(pcldf.loc[i].text)
  translated_texts = perform_translation([pcldf.loc[i].text], first_model, first_model_tkn)
  #print(translated_texts[0])
  back_translated_texts = perform_translation(translated_texts, second_model, second_model_tkn)
  #print(back_translated_texts[0])
  #replicated_pcls.iloc[]
  pcldf.iloc[i, 1] = back_translated_texts
  print(pcldf.iloc[i, 1])